# BENCHMARK RESNET
- https://github.com/kuangliu/pytorch-cifar

In [1]:
import os
import time
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class _ResNet(nn.Module):
    def __init__(self, block, num_blocks, input_size=64, num_classes=10):
        super(_ResNet, self).__init__()
        self.in_planes = input_size

        self.conv1 = nn.Conv2d(3, input_size, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(input_size)
        self.layer1 = self._make_layer(block, input_size, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [9]:
class ResNet(object):
    def __init__(self, params):
        
        super().__init__()
        self.params = params
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Instantiate model
        torch.manual_seed(params['random_seed'])
        np.random.seed(params['random_seed'])
        self.model = _ResNet(block=BasicBlock, 
                             num_blocks=[2, 2, 2, 2],
                             num_classes=params['n_classes'],
                             input_size=params['input_size']).to(self.device)

    def fit(self, train_loader, val_loader):
        
        params = self.params
        
        #------------------------------------- Optimization --------------------------------------#
        criterion = nn.CrossEntropyLoss()
        optimizer4nn = optim.SGD(self.model.parameters(), lr=params['initial_lr'],
                                 momentum=0.9, weight_decay=params['weight_decay'])
        scheduler = optim.lr_scheduler.StepLR(optimizer4nn, step_size=params['adjust_lr_step'], 
                                              gamma=params['lr_decay'])
        
        #---------------------------------------- Logging -----------------------------------------#
        step = 0
        epoch = 0
        break_flag = False
        self.best_acc = -1
        updated_optimizer = False
        trajectories = {'step':  [], 'epoch':  [],
                        'train_loss': [], 'train_acc': [],
                        'val_loss': [], 'val_acc': []}

        print('\n'+'='*43+' Fitting  ResNet '+'='*43)
        while step <= params['iterations']:
            # Train
            epoch += 1
            self.model.train()
            for batch_idx, (inputs, targets) in enumerate(train_loader):
                step+=1
                start = time.time()
                
                if break_flag: # weird epoch breaker
                    continue
                
                #--------------------------------- Forward and Backward ---------------------------------#
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                optimizer4nn.zero_grad()
                outputs = self.model(inputs)

                loss = criterion(outputs, targets)

                loss.backward()
                optimizer4nn.step()

                #----------------------------------- Evaluate metrics -----------------------------------#
                if (step % params['display_step']) == 0:
                    train_loss, train_acc = self.evaluate_performance(loader=train_loader, criterion=criterion)
                    val_loss, val_acc = self.evaluate_performance(loader=val_loader, criterion=criterion)
                    
                    display_str = f'step: {step} time: {time.time()-start:03.3f} ** '
                    display_str += f'train_loss: {train_loss:.4f} train_acc {train_acc:.2f} ** '
                    display_str += f'val_loss: {val_loss:.4f} val_acc: {val_acc:.2f}'
                    print(display_str)
                    
                    trajectories['train_loss'] += [train_loss]
                    trajectories['train_acc']  += [train_acc]
                    trajectories['val_loss']   += [val_loss]
                    trajectories['val_acc']    += [val_acc]
                    
                    if val_acc > self.best_acc:
                        self.best_acc = val_acc
                        
                if step > params['iterations']:
                    break_flag=True
                
                # Update optimizer learning rate
                scheduler.step()

        #---------------------------------------- Final Logs -----------------------------------------#
        print('\n'+'='*43+' Finished Train '+'='*43)
        self.train_loss   = trajectories['train_loss'][-1]
        self.train_acc    = trajectories['train_acc'][-1]
        self.val_loss     = trajectories['val_loss'][-1]
        self.val_acc      = trajectories['val_acc'][-1]
        self.trajectories = trajectories
        
    def evaluate_performance(self, loader, criterion):
        self.model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(loader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        acc = 100.*correct/total
        test_loss = test_loss/total
            
        self.model.train()
        return test_loss, acc
    
    def save_weights(self, path):
        torch.save(self.model.state_dict(), path)    

    def load_weights(self, path):
        self.model.load_state_dict(torch.load(path,
                                   map_location=torch.device(self.device)))
        self.model.eval()

# TEST RESNET

In [10]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1024, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [11]:
params = {'input_size': 64,
          'n_classes': 10,
          #'iterations': 50_000, #(n_samples/batch_size) * epochs
          'iterations': 1_000, #(n_samples/batch_size) * epochs
          'display_step': 500,
          'initial_lr': 0.1,
          'lr_decay': 0.5,
          'adjust_lr_step': 50_000//2,
          'weight_decay': 5e-4,
          'random_seed': 1}

model = ResNet(params)
model.fit(train_loader=trainloader, val_loader=testloader)


=========================================== Fitting  ResNet ===========================================
step: 500 time: 11.575 ** train_loss: 0.0122 train_acc 41.95 ** val_loss: 0.0015 val_acc: 42.85
step: 1000 time: 11.246 ** train_loss: 0.0091 train_acc 57.98 ** val_loss: 0.0012 val_acc: 57.80

=========================================== Finished Train ===========================================
